# *Pipelines* en *Scikit-Learn*

Este notebook es una adaptación del [original de *Aurélien Gerón*](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb), de su libro: [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 3rd Edition. Aurélien Géron](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/)

## Pasos previos

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

housing = pd.read_csv("./data/housing.csv") 
train_set, test_set = train_test_split(housing, test_size=0.2,
    stratify=pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5]),
    random_state=42
    )

housing = train_set.drop("median_house_value", axis=1) # Eliminamos la columna de la variable dependiente
housing_labels = train_set["median_house_value"].copy() # Guardamos la variable dependiente (etiquetas)

housing_num = housing.select_dtypes(include=[np.number]) # seleccionamos las columnas numéricas

## Pipelines de preprocesamiento

Un ***pipeline*** es una secuencia de componentes de procesado de datos. La clase 'Pipeline' de scikit-learn permite crear objetos que representen esta secuencias, para poder aplicarlas después a cualquier conjunto de datos.

Todos los **estimadores** excepto el último deben ser **transformadores**. Cuando llamamos al método `fit` de la clase 'Pipeline', se llama al método `fit_transform` de cada estimador secuencialmente, pasando la salida del método `transform` de un estimador al siguiente. El último estimador puede ser de cualquier tipo (transformador, clasificador, regresor, etc.).

Es importante tener claro qué signica cada [tipo de estimadores en scikit-learn](./sklearn/tipos_estimadores.md).

Vamos a construir un *pipeline* que prepreprocese los predictores numéricos.

El constructor de la clase 'Pipeline' recibe una lista de tuplas formadas por el nombre que identifica a cada estimador y ese estimador. Todos los estimadores han de ser **transformadores**, excepto el último, que puede ser cualquier tipo de estimador.

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")), # imputamos la mediana a los valores no disponibles
    ("standardize", StandardScaler()), # estandarizamos los valores
])
num_pipeline.steps

[('impute', SimpleImputer(strategy='median')),
 ('standardize', StandardScaler())]

También se puede usar la función ``make_pipeline``, que crea un *pipeline* como el anterior pero dando un nombre automáticamente a cada estimador.

In [3]:
from sklearn.pipeline import make_pipeline
num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
num_pipeline.steps

[('simpleimputer', SimpleImputer(strategy='median')),
 ('standardscaler', StandardScaler())]

El método `fit()` del *pipeline* llama al método `fit_transform()` de cada transformador, pasando la salida de cada uno al siguiente, y finalmente llama al método `fit()` del último estimador.
El método `fit_transform()` del *pipeline* hace lo mismo, pero llama al método `fit_transform()` del último estimador.


In [4]:
housing_num_prepared = num_pipeline.fit_transform(housing_num)
housing_num_prepared[:2].round(2)

array([[-0.94,  1.35,  0.03,  0.58,  0.64,  0.73,  0.56, -0.89],
       [ 1.17, -1.19, -1.72,  1.26,  0.78,  0.53,  0.72,  1.29]])

Para visualizar mejor lo que hace el *pipeline*, podemos volver a construir un dataframe con sus resultados.

In [5]:
pd.DataFrame(housing_num_prepared,
            columns=num_pipeline.get_feature_names_out(), # obtenemos nombres de columnas tras transformar
            index=housing_num.index).head(2) 

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
12655,-0.941350,1.347438,0.027564,0.584777,0.640371,0.732602,0.556286,-0.893647
15502,1.171782,-1.192440,-1.722018,1.261467,0.781561,0.533612,0.721318,1.292168


In [6]:
num_pipeline[1]

StandardScaler()

In [7]:
num_pipeline[:-1]

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median'))])

In [8]:
num_pipeline.named_steps["simpleimputer"]

SimpleImputer(strategy='median')

Con el método `set_params` podemos cambiar el valor de un parámetro de un estimador.

In [17]:
num_pipeline.set_params(simpleimputer__strategy="mean")

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('standardscaler', StandardScaler())])

Para poder crear un *pipeline* que preprocese además de los predictores numéricos, los categóricos, necesitamos un transformador que seleccione las columnas que queremos transformar. Scikit-learn proporciona la clase `ColumnTransformer` para esto.

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms",
               "total_bedrooms", "population", "households", "median_income"]
cat_attribs = ["ocean_proximity"]

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"), # imputamos la moda a los valores no disponibles
    OneHotEncoder(handle_unknown="ignore")) # codificamos las categorías

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs), # aplicamos el pipeline numérico a las columnas numéricas
    ("cat", cat_pipeline, cat_attribs)], # aplicamos el pipeline categórico a las columnas categóricas
    remainder="passthrough" # el resto de columnas se mantienen sin cambios
)

El parámetro `remainder='passthrough'` indica que las columnas que no se han seleccionado para transformar se pasarán directamente al *pipeline* final sin cambios. Si no se especifica, las columnas que no se han seleccionado se eliminarán (por defecto, `remainder='drop'`). En este caso, se están pasando todas, así que no habrá diferencia.

Para poder asignar pipelines a todas las columnas en función de su tipo, podemos usar la función `make_column_transformer`.

In [13]:
from sklearn.compose import make_column_selector, make_column_transformer

preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include=object)),
)

y podemos usar el método `fit_transform()` del *pipeline* para transformar los datos de entrenamiento.

In [14]:
housing_prepared = preprocessing.fit_transform(housing)

In [15]:
housing_prepared_fr = pd.DataFrame(
    housing_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=housing.index)
housing_prepared_fr.head(7).T

,12655,15502,2908,14053,20496,1481,18125
pipeline-1__longitude,-0.941350,1.171782,0.267581,1.221738,0.437431,-1.231094,-1.226099
pipeline-1__latitude,1.347438,-1.192440,-0.125972,-1.351474,-0.635818,1.085499,0.790817
pipeline-1__housing_median_age,0.027564,-1.722018,1.220460,-0.370069,-0.131489,-0.051963,-0.449595
pipeline-1__total_rooms,0.584777,1.261467,-0.469773,-0.348652,0.427179,-0.661977,0.747520
pipeline-1__total_bedrooms,0.638183,0.779415,-0.547672,-0.038752,0.270495,-0.688903,0.331371
pipeline-1__population,0.732602,0.533612,-0.674675,-0.467617,0.374060,-0.623583,0.324761
pipeline-1__households,0.556286,0.721318,-0.524407,-0.037297,0.220898,-0.652174,0.383269
pipeline-1__median_income,-0.893647,1.292168,-0.525434,-0.865929,0.325752,-0.094224,1.895358
pipeline-2__ocean_proximity_<1H OCEAN,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
pipeline-2__ocean_proximity_INLAND,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
preprocessing.get_feature_names_out()

array(['pipeline-1__longitude', 'pipeline-1__latitude',
       'pipeline-1__housing_median_age', 'pipeline-1__total_rooms',
       'pipeline-1__total_bedrooms', 'pipeline-1__population',
       'pipeline-1__households', 'pipeline-1__median_income',
       'pipeline-2__ocean_proximity_<1H OCEAN',
       'pipeline-2__ocean_proximity_INLAND',
       'pipeline-2__ocean_proximity_ISLAND',
       'pipeline-2__ocean_proximity_NEAR BAY',
       'pipeline-2__ocean_proximity_NEAR OCEAN'], dtype=object)